# Here, I build out the Langevin integrator capacities of my system. 

In [69]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import time

# built a random number generator courtesy of numpy
# rng = np.random.default_rng()


In [70]:
N= 10 # number of particles

p = np.empty((N,3),dtype=float) # momentum vector
q= p.copy() # position vector
M = np.empty((N,1),dtype=float) # mass vector
dt = .001 # integrator timestep
num_steps = 100 # total simulation time
T = 10.0 # temperature
gamma = .5 # friction
kB = 1.e-1 # what should this be??


potential_type = "harmonic"
kSpring=1.0



One can choose numerous potential energy functions. These describe how the (relative) positions of particles affect the energy of the system. Because we are running a Langevin integrator which is grounded in Newtonian dynamics, we care about forces and their effects on momenta. However, it is often convenient to keep track of potentials, as they are usually relatively simple functions of the position vector of a system. Depending on the integrator used, many scientific applications allow you to design potentials symbolically, eg `Spring_Potential = ''0.5*k*x^2''`. These integrators then have tables, or symbolic differentiators, to differentiate the potential into a force. This project won't attempt to do that; instead, I'll build functions that get $U$ and $\nabla U$ directly depending on the type of potential specified. 

Perhaps the simplest potential is a harmonic potential, $U(x) = \frac{1}{2}kx^2.$ Observe that $-D_xU = F_x = -kx $, where the subscript $x$ denotes the x component or differentiating in the $x$ direction. For our harmonic potentials, we can imagine a spring between each pair of particles. We thus sum the pairwise potentials for each bead: $$U_{\mathrm{sys}} = \sum_{i < j}\frac{1}{2}k||q_i-q_j||^2 = \frac{1}{2}k \sum_{i<j}||q_i-q_j||^2.$$ To take the gradient, we differentiate wrt each component of $q$: $$(\nabla U)_i = D_{q_i}U = \frac{1}{2}k\sum_{j \neq i} D_{q_i}||q_i-q_j||^2 = k\sum_{j \neq i} (q_i-q_j).$$ We lose most of the terms because they are constant wrt $q_i$, and we lose the magnitude/absolute value at the end because because we can think of this as differentiating $\mathbf{r} \cdot \mathbf{r}$. 

In [71]:
# implement our harmonic potentials

def get_harmonic_U(k):
    def U(q):
        difs = q[:,None,:] - q[None,:,:] # Here, we create a matrix of differences q_i - q_j
        sumSquares = np.sum(difs**2,axis=2) # sum the x,y, and z components to get an N x N matrix

        return(0.5*k*np.sum(np.triu(sumSquares,k=1))) # we only care about the unique pairs
    return(U) 

def get_harmonic_Nabla(k):
    def Nabla(q):
        difs = q[:,None,:] - q[None,:,:] # Here, we create a matrix of differences q_i - q_j
        return(k* np.sum(difs,axis = 1)) # sum over j, noting that the dif for j=i is 0
    return(Nabla) 


We want to apply B A O A B. In the README.md file attached, we describe these terms in more detail. The important thing to note here is that A depends on p and is applied to q, while B depends on q and is applied to p. The beauty of BAOAB is that it allows us to freeze q wrt p and p wrt q for the A and B steps by examining variable-specific flows. Hence, integrating these operations over dt is equivalent to integrating a constant. In particular, we have $$\int \frac{dq}{dt}dt = \int \frac{p}{M}dt \implies \Delta q = \Delta t\frac{p}{M} $$ with a similar result for $B$. However, the $O$ operation is applied to $p$ and depends on $p$. We have $$\int \frac{dp}{dt} = \int -\gamma p + \xi(t) \sqrt{2\gamma k_B Tm} \ dt.$$ Letting $\sigma = \sqrt{2\gamma k_B Tm}$ and $\xi(t)dt = W_t$, a vector of $N$ independent Wiener processes. Then we use a technique common in the soft sciences (physics, eg.) and multiply by $e^{\gamma t}$ to get $$\begin{aligned}  d(e^{\gamma t}p) = \gamma e^{\gamma t}p dt + e^{\gamma t}dp = \gamma e^{\gamma t}pdt + e^{\gamma t}(-\gamma p dt + \sigma d W_t)  = \sigma e^{\gamma t}dW_t \end{aligned}.$$ We can then integrate as physicists, getting $$   \int d(e^{\gamma t}p) = \sigma \int e^{\gamma t} dW_t  \implies e^{\gamma (t + dt)}p(t+dt) - e^{\gamma t}p(t) = \sigma \int _t ^{t + dt} e^{\gamma t} dW_s$$ where the RHS is a bizarre integral over our stochastic function. Then simple algebra gives $$  p(t + dt) = e^{-\gamma dt}p(t) + \sigma \int_t^{t + dt} e^{\gamma (s-t-dt)}dW_s.$$

How do we evaluate that integral? This requires some stochastic calculus beyond the scope of this writeup. See https://en.wikipedia.org/wiki/It%C3%B4_calculus. In particular, we apply the Ito isometry $$\mathbb{E}[(\int_0^{t}g(s))^2]= \mathbb{E}[\int_0^{t}(g(s))^2]$$and, observing that we're integrating a Brownian motion process and so get 0 mean, we have the variance for free (that is, we subtract off the square of the expectation value of our integral), arriving at $$\mathrm{Var}= \int_t^{t+dt}(\sigma e^{\gamma(s-t-dt)})^2ds,$$ which is easily evaluated; we let $a = e^{-\gamma dt}$, and arrive at $$p(t+dt) = ap(t)+ \sqrt{mk_B T(1-a^2)}\xi,$$ where again $\xi \sim \mathscr{N}(0,1)$ and the coefficient before it is the standard deviation we arrived at in the integral above (recall that $\mathrm{Var}(ax) = a^2\mathrm{Var}(x)).$

Hence, to apply our 'O' step, we need only scale $p$ by a factor of $a$ and add a normally distributed kick with the appropriate scaling. Unfortunately, `1-((e**x)**2)` is quite numerically unstable; for very small (on the order of dt) x, `e**x` can get uncomfortably close to 1; squaring can become a mess, and then you risk catestrophic cancellation subtracting two numbers close to 1. Luckily numpy has plenty of built in tools for this. 

In [72]:
def apply_A(p,q,dt,M=M):
    # the A operator doesn't touch p, but it adjusts q (position) via Newton
    q += p/M * dt # careful! numpy will broadcast M.shape = (N,1) but not (N,)... need to keep an eye out
    return(p,q)

def apply_B(p,q,dt,getNablaU):
    # the A operator applies the newtonian force kick 
    F = -getNablaU(q)
    p += F*dt
    return(p,q)

def apply_O(p,q,dt,gamma=gamma,M=M,kB=kB,T=T):
    # the O operator adds thermal randomness to our system
    # it only adjusts p

    # a = np.exp(-gamma*dt)
    # stdev = np.sqrt(M*kB*T*(1-a**2)) # NOTE: probably want to make this more stable later on
    ## when dt is small, a is close to 1, so 1-a**2 is close to 0; this can make the above unstable
    ## Hence, use the following: 

    a = np.exp(-gamma*dt)
    one_minus_a2 = np.expm1(-2*gamma*dt) * (-1)   # equals 1 - a**2 with better accuracy
    one_minus_a2 = np.clip(one_minus_a2, 0.0, 1.0) # clip to avoid more floating point errors
    stdev = np.sqrt(M*kB*T*(one_minus_a2))
    xi = rng.normal(size = p.shape)
    p = a*p + stdev * xi

    return(p,q)

## Main Function to Run the Simulation

In [73]:
def run(p0 = p,q0=q,N=N,M=M,dt=dt,num_steps=num_steps,T=T,gamma=gamma,kB=kB,
        potential_type = potential_type,
        timing = False, printing_steps = False):
    other_data = {}
    # perform BAOAB


    if potential_type == "harmonic":
        getNabla = get_harmonic_Nabla(kSpring)
        getU = get_harmonic_U(kSpring)

    p=p0
    q=q0

    q_table,p_table,U_table = [q.copy()],[p.copy()],[getU(q)]
    if timing: 
        start = time.perf_counter()
    for step in range(num_steps):
        if printing_steps and step>0:
            if step % int(num_steps//10) == 0:
                print(f"At step {step} out of {num_steps}")
        p,q = apply_B(p,q,dt/2,getNabla)
        p,q = apply_A(p,q,dt/2,M)
        p,q = apply_O(p,q,dt,gamma,M,kB,T)
        p,q = apply_A(p,q,dt/2,M)
        p,q = apply_B(p,q,dt/2,getNabla)

        q_table.append(q.copy())
        p_table.append(p.copy())
        U_table.append(getU(q))
    if timing: 
        end = time.perf_counter()
        other_data['time'] = end-start
    
    return (np.array(p_table),np.array(q_table),np.array(U_table),other_data)


# Experiment

### Basic First Experiment

In [74]:
# First experiment: reduced units

rng = np.random.default_rng(42)


N= 10**2 # number of particles

p = np.empty((N,3),dtype=float) # momentum vector
q= p.copy() # position vector
M = np.ones((N,1),dtype=float) 
dt = 1.e-3
num_steps = 10**3 
T = 1.0
gamma = 1.0
kB = 10 

# starting positins in a small Gaussian, then center at (0,0,0)
q = 0.1 * rng.normal(size=(N,3))
q -= q.mean(axis=0, keepdims=True)

# starting momenta are boltzmann distributed with stdev = sqrt(mkBT) per component
p_std = np.sqrt(M * kB * T)         
p0 = p_std * rng.normal(size=(N,3))
# here, we 'center' the momenta so they're not biased in a given direction
p -= p.mean(axis=0, keepdims=True)
p0,q0 = p,q

potential_type = "harmonic"
kSpring=1.0

pTable,qTable,UTable,other_Data = run(p0,q0,num_steps=num_steps,
                           N=N,M=M,dt=dt,gamma=gamma,kB=kB,T=T,
                           potential_type="harmonic",
                           timing = True,printing_steps=True)

print(f"other data = {other_Data}")

At step 100 out of 1000
At step 200 out of 1000
At step 300 out of 1000
At step 400 out of 1000
At step 500 out of 1000
At step 600 out of 1000
At step 700 out of 1000
At step 800 out of 1000
At step 900 out of 1000
other data = {'time': 0.5644838749431074}


#### Some Simple Timing Experiments

In [ ]:
timing_experiments = False
if timing_experiments:
    time_table_in_N = []
    Ns = []
    for exp in np.arange(1,6,0.1):
        N_exp = 10**exp
        print(f"N = {N_exp}")
        Ns.append(exp)
        _,_,_,o = run(p0,q0,num_steps=num_steps,
                            N=N_exp,M=M,dt=dt,gamma=gamma,kB=kB,T=T,
                            potential_type="harmonic",
                            timing = True,printing_steps=False)
        time_table_in_N.append(o['time'])

    plt.plot(Ns,time_table_in_N)
    plt.title('Timing vs N')
    plt.xlabel('N')
    plt.ylabel('Time (s)')
    plt.show()

N = 10.0
N = 12.589254117941675
N = 15.848931924611142
N = 19.952623149688808
N = 25.11886431509582
N = 31.622776601683825
N = 39.81071705534978
N = 50.1187233627273
N = 63.09573444801943
N = 79.4328234724283
N = 100.0000000000002
N = 125.89254117941701
N = 158.48931924611173
N = 199.5262314968885
N = 251.18864315095874
N = 316.2277660168389
N = 398.1071705534986
N = 501.18723362727405
N = 630.9573444801956
N = 794.3282347242846
N = 1000.0000000000041
N = 1258.9254117941725
N = 1584.8931924611206
N = 1995.262314968889
N = 2511.8864315095925
N = 3162.2776601683954
N = 3981.0717055349937
N = 5011.8723362727505
N = 6309.573444801968
N = 7943.282347242862
N = 10000.000000000062
N = 12589.254117941764
N = 15848.931924611239
N = 19952.62314968891
N = 25118.864315095976
N = 31622.77660168405
N = 39810.71705535002
N = 50118.72336272756
N = 63095.734448019815
N = 79432.82347242886
N = 100000.00000000081
N = 125892.54117941765
N = 158489.31924611272
N = 199526.23149688993
N = 251188.64315096027
